In [0]:
# Thư viện
from pyspark.sql import functions as F
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
# Thông số ban đầu
table_input  = "vaccine_order_sampled"    
table_region = "dim_shop"                       
table_output = "vaccine_cluster_result"   

random_state = 42                         

In [0]:
# Đọc dữ liệu gốc
df_orders = spark.table(table_input)
df_region = spark.table(table_region)

print("Dữ liệu đầu vào:")
display(df_orders.limit(5))
print("Dữ liệu thông tin vùng:")
display(df_region.limit(5))

In [0]:
# Khám phá - Phân tích dữ liệu
# Tổng số dòng, số trung tâm, số tháng
summary_stats = df_orders.select(
    F.countDistinct("order_code").alias("distinct_orders"),
    F.countDistinct("shop_code").alias("distinct_shops"),
    F.countDistinct("month").alias("distinct_months")
).collect()[0]

print(f"Tổng quan bộ dữ liệu:")
print(f"- Số đơn hàng: {summary_stats['distinct_orders']:,}")
print(f"- Số trung tâm tiêm: {summary_stats['distinct_shops']:,}")
print(f"- Số tháng: {summary_stats['distinct_months']}")

# Thống kê giá trị bán hàng
df_stats = df_orders.select(
    F.mean("line_item_amount_after_discount").alias("mean_amount"),
    F.percentile_approx("line_item_amount_after_discount", 0.5).alias("median_amount"),
    F.max("line_item_amount_after_discount").alias("max_amount")
)

display(df_stats)

# Phân bố số lượng vaccine theo tháng
df_monthly = (
    df_orders.groupBy("month")
    .agg(F.countDistinct("order_code").alias("total_orders"))
    .orderBy("month")
)

print("Số lượng đơn hàng theo tháng:")
display(df_monthly)

In [0]:
# Tiền xử lí dữ liệu

# Tổng hợp dữ liệu theo cấp đơn hàng
df_order_level = (
    df_orders.groupBy("order_code", "shop_code", "month")
    .agg(
        F.sum("line_item_quantity").alias("num_items"),
        F.sum("line_item_amount_after_discount").alias("total_amount"),
        F.countDistinct("sku").alias("num_unique_products"),
        F.sum(F.when(F.col("line_item_discount_promotion") > 0, 1).otherwise(0)).alias("promotion_flag")
    )
)

print("Dữ liệu sau khi tổng hợp theo đơn hàng:")
display(df_order_level.limit(5))

In [0]:
# Gắn thông tin vùng địa lí
df_joined = df_order_level.join(
    df_region.select("shop_code", "region_name"),
    on="shop_code",
    how="left"
)

print("Dữ liệu sau khi thêm vùng địa lí:")
display(df_joined.limit(5))

In [0]:

pdf = df_joined.toPandas()
pdf = pdf.drop(columns=["shop_code", "order_code"], errors="ignore")
pdf["total_amount_log"] = np.log1p(pdf["total_amount"])  

# Danh sách các biến
numeric_features = ["num_items", "total_amount_log", "num_unique_products", "promotion_flag"]
categorical_features = ["region_name"]

# Chuẩn hóa và mã hóa
preprocessor = ColumnTransformer(transformers=[
    ("num", StandardScaler(), numeric_features),
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
])
X_prepared = preprocessor.fit_transform(pdf)

In [0]:
# Thiết lập biểu đồ phân phối của total_amount

# Thiết lập style cho biểu đồ
sns.set_style("whitegrid")

# Tạo một figure chứa hai biểu đồ con đặt cạnh nhau
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Phân phối của total_amount gốc
sns.histplot(pdf['total_amount'], kde=True, ax=axes[0], bins=50)
axes[0].set_title('Phân phối của total_amount (Trước khi biến đổi)', fontsize=14)
axes[0].set_xlabel('Giá trị đơn hàng')
axes[0].set_ylabel('Tần suất')

# Phân phối của total_amount sau khi biến đổi logarit
sns.histplot(pdf['total_amount_log'], kde=True, ax=axes[1], color='seagreen')
axes[1].set_title('Phân phối của total_amount (Sau khi biến đổi logarit)', fontsize=14)
axes[1].set_xlabel('Log(1 + Giá trị đơn hàng)')
axes[1].set_ylabel('Tần suất')

In [0]:
# Xác định số lượng cụm tối ưu
inertia = []
K = range(2, 10)
for k in K:
    km = KMeans(n_clusters=k, random_state=random_state)
    km.fit(X_prepared)
    inertia.append(km.inertia_)

plt.figure(figsize=(8,5))
plt.plot(K, inertia, marker='o')
plt.title("Elbow Method - Xác định số cụm tối ưu")
plt.xlabel("Số cụm (K)")
plt.ylabel("Inertia (Tổng khoảng cách nội cụm)")
plt.grid(True)
plt.show()

In [0]:
# Phân cụm bằng K-Means
optimal_k = 6  
kmeans = KMeans(n_clusters=optimal_k, random_state=random_state)
kmeans.fit(X_prepared)
pdf["cluster"] = kmeans.labels_

print("Đã phân cụm xong. Số cụm:", optimal_k)
pdf.head()

In [0]:



# Dữ liệu đầu vào đã được chuẩn hóa và mô hình K-Means đã huấn luyện trước đó
labels = kmeans.labels_

# Tính chỉ số đánh giá
sil_score = silhouette_score(X_prepared, labels)

# Hiển thị kết quả đánh giá
print("=== ĐÁNH GIÁ MÔ HÌNH PHÂN CỤM K-MEANS ===")
print(f"Số cụm (K): {optimal_k}")
print(f"Silhouette Score: {sil_score:.4f}")

# So sánh Silhouette theo từng cụm
sample_silhouette_values = silhouette_samples(X_prepared, labels)
y_lower = 10
plt.figure(figsize=(8, 6))
for i in range(optimal_k):
    ith_cluster_silhouette_values = sample_silhouette_values[labels == i]
    ith_cluster_silhouette_values.sort()
    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i
    plt.fill_betweenx(np.arange(y_lower, y_upper),
                      0, ith_cluster_silhouette_values)
    plt.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
    y_lower = y_upper + 10  # khoảng cách giữa các cụm

plt.title("Biểu đồ Silhouette cho từng cụm")
plt.xlabel("Giá trị hệ số Silhouette")
plt.ylabel("Các cụm")
plt.axvline(x=sil_score, color="red", linestyle="--")
plt.show()


In [0]:
# Phân tích kết quả phân cụm

# Tìm đặc trưng trung bình của từng cụm
cluster_summary = (
    pdf.groupby("cluster")[["num_items", "total_amount", "num_unique_products", "promotion_flag"]]
    .mean()
    .round(2)
    .reset_index()
)

print("Đặc trưng trung bình của từng cụm:")
display(cluster_summary)

In [0]:
# Phân tích phân bố cụm theo khu vực

# Thống kê số lượng cửa hàng thuộc từng cụm trong mỗi khu vực
cluster_summary = (
    pdf.groupby(["region_name", "cluster"])
       .size()
       .reset_index(name="count")
       .sort_values(["region_name", "cluster"])
)

print("Phân bố cụm theo khu vực:")
display(cluster_summary)

# Vẽ biểu đồ trực quan hóa phân bố
plt.figure(figsize=(10,6))
sns.barplot(data=cluster_summary, x="region_name", y="count", hue="cluster")
plt.title("Phân bố cụm theo khu vực")
plt.xlabel("Khu vực")
plt.ylabel("Số lượng cửa hàng / đơn hàng")
plt.legend(title="Cụm")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()